## Import modules

In [52]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 
import random
import math
import operator
from collections import Counter


## Standard Hough Transformation Function

In [53]:
def houghTransform(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # kernel = np.ones((15, 15), np.uint8)

    # opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
    # gray = cv2.medianBlur(gray,3) # Removes salt and pepper noise by convolving the image with a (3,3) square kernel
    # gray = cv2.GaussianBlur(gray, (9,9), 0)  # Smoothens the image by convolving the with a (9,9) Gaussian filter 
    blur= cv2.GaussianBlur(gray,(7,7),1)             # Apply Gaussian Blur with kernel size of (7,7)

    # Take the median of the image and calculate the upper and lower bound
    v =  np.median(blur)
    lower =  int(max(0 , (1.0 - 0.33) * v))
    upper = int(min(255, (1.0 + 0.33) * v))
    edges = cv2.Canny(blur, lower, upper, apertureSize=3)

    cv2.imwrite("edges.jpg", edges)
    
    # Compute the Hough Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 150)
    print("Found lines: {}".format(len(lines)))
    houghLines = []
    # Draw the lines
    for line in lines:
        rho, theta = line[0]
        if abs(theta-np.pi/90) < np.pi/9:
            continue
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        line_points = list(((x1,y1), (x2,y2)))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),1)
        cv2.imwrite("StandardHough.jpg", img);
        # print("line_points: ", line_points)
        houghLines.append( line_points)
        # cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    return houghLines

## Probablistic Hough Transform

In [54]:
def probabilistichoughTransform(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # kernel = np.ones((15, 15), np.uint8)

    # opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
    gray = cv2.medianBlur(gray,3) # Removes salt and pepper noise by convolving the image with a (3,3) square kernel
    gray = cv2.GaussianBlur(gray, (9,9), 0)  # Smoothens the image by convolving the with a (9,9) Gaussian filter 
    #Find Image Gradient along the Y-axis
    sobely_64f = cv2.Sobel(gray, cv2.CV_64F, 0, 1 , ksize = 3) # Sobel filter size:(3,3)
    abs_sobelx64F = np.absolute(sobely_64f)
    abs_sobelx64F = abs_sobelx64F/ abs_sobelx64F.max() * 255 # Force pixel values to be in range [0, 255] 
    sobely_8u = np.uint8(abs_sobelx64F) # Force pixels to take integer values
    y_grad_threshold = sobely_8u.copy() # Make a copy for further thresholding 
    

    # Find Image Gradient along the X-axis
    sobelx_64f = cv2.Sobel(gray, cv2.CV_64F, 1 , 0,ksize = 3) # Sobel filter size: (3,3)
    abs_sobelx = np.absolute(sobelx_64f)
    abs_sobelx = abs_sobelx/abs_sobelx.max() *  255 # Force pixel values to be in range [0, 255]
    sobelx_8u = np.uint8(abs_sobelx)

    # Find gradient direction
    grad_directions = np.arctan2(sobely_8u, sobelx_8u) 
    # Threshold gradient direction image to process essential pixels only
    y_grad_threshold[y_grad_threshold > 30] = 255
    grad_directions[y_grad_threshold < 10] = -np.pi 
    grad_directions_normalized = ((grad_directions*180/np.pi) + 180)/360 # Force pixels to be in range [0,1]
    grad_directions_8u = np.uint8(grad_directions_normalized * 255) # Rescale pixels to lie in [0, 255]
    
    # edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # cv2.imwrite("edgesProba.jpg", edges)
    # Hough Transform
    maxLineGap = 5 # max pixel spacing between 2 lines for it to be considered 1 line
    adaptive = int(75*1) # Needs to be decreased as the resolution is decreased
    minLineLength = adaptive # Min length of line in pixels to be extracted

    # Use the probabilistic Hough Transform with 'rho' value of 1 pixel and 'theta' value of 1 degree
    # The values of 'rho' and 'theta' represent the granularity of the detection scheme (rho = 1 means check every pixel, theta =  1 degree means check for every 1 degree variation) and 
    # determine the dimensions of the Hough accumulator array (please see OpenCV Hough Transform for more details) 
    # You can speed up line detection (by sacrificing for accuracy) by increasing the rho and theta values     
    # The probabilistic Hough transform randomly samples sections of the image for lines (this only works if the image is structurally rich). 
    # The output is a list in which every element describes a line by its end points (x1,y1) and (x2,y2) 
    lines = cv2.HoughLinesP(grad_directions_8u,1,np.pi/180,350,minLineLength,maxLineGap) # Threshold of 400 needs to be decreased with image resolution

    # Compute the Hough Transform
    print("Found lines: {}".format(len(lines)))
    houghLinesP =  []
    formatted_hlp = []
    for i,line in enumerate(lines):
        # print(i)
        x1, y1, x2, y2 = line[0]
        line_points = list(((x1,y1), (x2,y2)))
        houghLinesP.append( line_points)
        formatted_hlp.append((x1,y1,x2,y2))
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 1)
        cv2.imwrite("StandardHoughProb.jpg", img);

    return houghLinesP, formatted_hlp


In [55]:

def determine(a, b):
    return a[0] * b[1] - a[1] * b[0]

def distancebetweenTwoPoints(p1, p2):
    '''
    calculate distance between point tuples
    '''
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
def checklineIntersection(line, p1, p2):
    '''
    check if a line passes throguh two points
    '''
    length1 = distancebetweenTwoPoints((line[0][0],line[0][1]),(p1, p2))    
    length2 = distancebetweenTwoPoints((line[1][0],line[1][1]),(p1, p2))
    length3 = distancebetweenTwoPoints((line[0][0],line[0][1]),(line[1][0],line[1][1]))

    if int(length1) + int(length2)  == int(length3):
        return True
    else:
        return False
def intersectionOftwoLines(line_1,line_2, img_h, img_w):
    '''
    finds the intersection point between two lines
    '''

    xVal = (line_1[0][0] - line_1[1][0], line_2[0][0] - line_2[1][0])
    yVal = (line_1[0][1] - line_1[1][1], line_2[0][1] - line_2[1][1])

    intersect = determine(xVal, yVal)

    if intersect == 0:
        return 0,0

    temp = (determine(*line_1), determine(*line_2))
    x = determine(temp, xVal) / intersect
    y = determine(temp, yVal) / intersect

    if x <=0 or y<=0 or x>=img_w or y>=img_h:
        return img_w//2 , img_h//2
    return int(x), int(y)

def intersectionofmultipleLines(lines, img_h, img_w):
    '''
    find all pairs of lines that intersect
    '''
    intersections = []
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            # print(lines[i])
            # print(lines[j])
            intersection = intersectionOftwoLines(lines[i], lines[j],img_h, img_w)
            if intersection not in intersections:
                intersections.append(intersection)
    return intersections

In [56]:
def twoLineVP(img, houghLines, color , functype):
    '''
    function to get the Vanishing point using two lines
    '''
    ## TO-DO : make sure points are not negative
    countInt = {}
    i_h = int(img.shape[0])
    i_w = int(img.shape[1])
    for i,line1 in enumerate(houghLines):
        for j,line2 in enumerate(houghLines):
            # print("-- ",i,j)
            # if both the lines are same then don't process
            if line1 == line2:
                continue
            
            # check if point the lines are intersecting
            # print(line1, line2)
            # print(intersectionOftwoLines(line1, line2))
            pointX, pointY = intersectionOftwoLines(line1, line2, i_h, i_w)
            if pointX < 0 or pointY < 0 or pointX > i_w or pointY > i_h:
                continue
            countInt[(pointX, pointY)] = 1

            # now check for all other lines that intersect with this pair
            for lines in houghLines:
                # if lines are same then don't process
                if lines == line1 or lines == line2:
                    continue

                # check if the line is passing through the pair of points
                flag =  checklineIntersection(lines, pointX, pointY)
                if flag:
                    countInt[(pointX, pointY)] += 1


    finalInt = max(countInt.items(), key=operator.itemgetter(1))[0]
    print("Coordinates of VP : ")
    print(finalInt)
    cv2.circle(image, (int(finalInt[0]), int(finalInt[1])), int(image.shape[1]//60), (0, 255, 0), 3)
    cv2.imwrite("LinePairVP"+functype+".jpg", image)
    return finalInt

In [57]:
# Function to get the Vanishing point using Least Square method for all the lines
def calculateError(A, x, b):
    return np.subtract(np.dot(A,x), b)
def leastSquareVP(img, houghLinesArrary, htype = "-std-"):
    A = [[0 for y in range(2)] for x in range(len(houghLinesArrary))]
    b = [[0 for y in range(1)] for x in range(len(houghLinesArrary))]
    count = 0
    for lines in houghLinesArrary:

        # Removing the Vertical and Horizontal lines
        if lines[0][0] == 0 or lines[1][0] == 0 or lines[1][0] - lines[0][0] == 0:
            lineSlope = 1
            lineAngle = 0
            continue
        else:
            lineSlope = (lines[1][1] - lines[0][1])/(lines[1][0] - lines[0][0])
            lineAngle = np.absolute(np.arctan(lineSlope)*180.0 / np.pi)

        # Check for Vertical Lines
        if lineAngle > 85 and lineAngle < 95:
            continue
        # Check for Horizontal Lines
        if lineAngle >= 0 and lineAngle < 10:
            continue
        
        # Convert the line to a*x + b*y = c form 
        A[count][0] = -(lines[1][1] - lines[0][1])                              # -(y2-y1)
        A[count][1] = (lines[1][0] - lines[0][0])                               # x2-x1
        b[count][0] = A[count][0] * lines[0][0]  +  A[count][1] * lines[0][1]   # -(y2-y1)*x1 + (x2-x1)*y1
        count += 1

    x = [[0 for y in range(1)] for x in range(2)]
    error_array = [[0 for y in range(1)] for x in range(len(houghLinesArrary))]
    solution = [[0 for y in range(1)] for x in range(2)]
    error_value = 9999999999.1
    aTemp = [[0 for y in range(2)] for x in range(2)]
    bTemp = [[0 for y in range(1)] for x in range(2)]

    # iterate through the rows
    for i in range(count):
        for j in range(count):
            
            if i >= j:
                continue

            # Store the values of A & b in temp 
            aTemp[0] = A[i]
            aTemp[1] = A[j]
            bTemp[0] = b[i]
            bTemp[1] = b[j]

            # check if the rank of A temp matrix is 2, if not skip
            if not np.linalg.matrix_rank(aTemp) == 2:
                continue

            # Calculate A*x = b, get x value by passing A & b
            x = np.linalg.solve(aTemp, bTemp)
            # print("x = ", x)
            if x[0] <=0 or x[1] <=0 or x[0] >= img.shape[1] or x[1] >= img.shape[0]:
                continue

            if len(x) == 0 or len(x[0]) == 0:
                continue

            # Calculate error assuming perfect intersection [A * x - b]
            error_array = calculateError(A, x, b);
            error_array = error_array/1000              # reducing error

            tempError = 0

            # sum up all the error values
            for i in range(len(error_array)):
                tempError += error_array[i][0] * error_array[i][0] / 1000

            tempError = tempError / 1000000

            # Check if current errorValue is less than minimum error, if so then update the solution & error value
            if(error_value > tempError):
                solution = x
                error_value = tempError

    # plot the value of solution using a circle to get an approx. vanishing point
    print("solution : ", solution)
    cv2.circle(img, (int(solution[0][0]), int(solution[1][0])), 50,(0, 0, 255), 10)
    cv2.imwrite("LeastSquareStandardHough"+htype+".jpg", img);

In [58]:
def filterHoughLines_ (img,lines, vpX = 0, vpY = 0):
    i_x = vpX # x-coordinate of the vanishing point
    i_y = vpY  # y-coordinate of the vanishing point
    i_h = int(img.shape[0]) # image height
    i_w = int(img.shape[1] * 1) # image width 


    # Intailizing empty lists
    lines_r_l = []

    # Yellow line to visualize vanishing point change with lateral camera movement
    # cv2.line(img,(0,int(i_h/(3))),(i_w,int(i_h/(3))),(0,255,255),2)

    # Select lines that satisfy application constraints (detecting lines ina an indoor corridor)
    if lines is not None: 
        for line in lines:
            x1, y1, x2, y2 = line 
            # if x1 <= 0 or y1 <= 0 or x1 >= i_w or y1 >= i_h:
            #     continue
            # if x2 <= 0 or y2 <= 0 or x2 >= i_w or y2 >= i_h:
            #     continue
            if (x1- x2)!=0: # Make sure lines are not collinear 
                theta = np.arctan2((y2-y1),(x2-x1))
                m2= np.tan(theta)
                l_mag = np.sqrt(np.square(x1 - x2) + np.square(y1 - y2))

                # Extend the lines to the entire image and compute the intersetion point
                c2 = y1 - m2*x1
                x3 = int(((i_w/2)+20) + x1) # 1000 was chosen arbitrarily (any number higher than half the image width) 
                y3 = int(m2*x3 + c2)
                x4 = int(x1 - ((i_h/2)+20)) # 1000 was chosen arbitrarily 
                y4 = int(m2*x4 + c2)
                # if x3 <= 0 or y3 <= 0 or x3 >= i_w or y3 >= i_h:
                #     continue
                # if x4 <= 0 or y4 <= 0 or x4 >= i_w or y4 >= i_h:
                #     continue
                if abs(theta) > 0.1 and l_mag > 20  and abs(theta) < 1.3: # Eliminate short lines and lines that are close to being perfectly horizontal or vertical
                    line_points = list(((int(x3),int(y3)), (int(x4),int(y4))))
                    lines_r_l.append(line_points)
                    cv2.line(img,(x3,y3),(x4,y4),(150,0,255),1)  
                    cv2.imwrite("FilteredHoughP.jpg", img);

        print("Found lines: {}".format(len(lines_r_l)))                      
        return lines_r_l
                      
    else:
        return lines # Since there were no measurement for this frame, return state info twice

In [59]:
def computeHomologousRepresentation(lines):
    '''
    Compute the Homogenous representation of the lines given the detected end points 
    '''
    # Create a matrix of zeros
    homoLines = np.zeros((len(lines),4))
    # Iterate through the lines and fill the homoLines matrix
    for i,line in enumerate(lines):
        # Get the end points of the line
        p1 = line[0]
        p2 = line[1]
        x1,y1 = p1
        x2,y2 = p2
        # Compute the slope and intercept
        m = (y2-y1)/(x2-x1)
        b = y1 - m*x1
        # Fill the homoLines matrix
        homoLines[i] = [m,b,1,0]
    return homoLines


def computeIntersection(lines):
    '''
    Compute the intersection of the lines given the homogenous representation
    '''
    # Create a matrix of zeros
    intersection = np.zeros((len(lines),2))
    # Iterate through the lines and fill the homoLines matrix
    for i,line in enumerate(lines):
        # Get the end points of the line
        x1,y1,x2,y2 = line.flatten()
        # Compute the slope and intercept
        m = (y2-y1)/(x2-x1)
        b = y1 - m*x1
        # Fill the homoLines matrix
        intersection[i] = [m*x1 + b, y1]
    return intersection
    
def crossProduct(x,y):
	res = np.dot(np.matrix([[0,-x[2],x[1]],[x[2],0,-x[0]],[-x[1],x[0],0]]),np.array(y).T)
	return np.array(res)

def findIntersectionfromHomogenousRepresentationsofLines(lines):
    '''
    given a list of lines, represented in homogenous form with slope and intercept, find the intersection point
    '''
    # Create a matrix of zeros
    new_int = []
    intersection = np.zeros((len(lines),2))
    # Iterate through the lines and fill the homoLines matrix
    for i,line in enumerate(lines):
        # Get the end points of the line
        x1,y1,x2,y2 = line.flatten()
        point_1 = np.array([x1,y1, 1])
        point_2 = np.array([x2,y2, 1])
        l = crossProduct(point_1,point_2)
        new_int.append(l)
        # Compute the slope and intercept
        m = line[0]
        b = line[1]
        # Fill the homoLines matrix
        intersection[i] = [int(m*x1+b),int(m*x2+b)]
    # return intersection
    return new_int, intersection

In [60]:
# image = cv2.imread(imageName)
# houghLines = houghTransform(image)
# houghLinesP, hlp  = probabilistichoughTransform(image)
# vpp_x,vpp_y = twoLineVP(image, houghLines,"-standard-")
# print("--------------------------------------------")
# vp_x, vp_y = twoLineVP(image, houghLinesP,"-probabilistic-")
# print("--------------------------------------------")


## Driver Code

In [ ]:
import random
imageName = "wrench.jpeg"
image = cv2.imread(imageName)
houghLines = houghTransform(image)
vpp_x,vpp_y = twoLineVP(image, houghLines,(255, 100, 100),"-standard-")

print("------------------------------------1----------------------------------------------------")
image = cv2.imread(imageName)
houghLinesP, hlp  = probabilistichoughTransform(image)
vp_x, vp_y = twoLineVP(image, houghLinesP,(255, 100, 100),"-probabilistic-")
print("------------------------------------2----------------------------------------------------")
image = cv2.imread(imageName)
leastSquareVP(image, houghLines)
image = cv2.imread(imageName)
leastSquareVP(image, houghLinesP,"-prob-")
print("------------------------------------22----------------------------------------------------")
print("vpp_x: ", vpp_x, " vpp_y: ", vpp_y)
print("vp_x: ", vp_x, " vp_y: ", vp_y)
##################################################################################################

image = cv2.imread(imageName)
filterHoughLinesstd = filterHoughLines_(image,hlp )
x_fs,y_fs = twoLineVP(image, filterHoughLinesstd,(100, 100, 100),"-filtered-")
print("x_fs : ", x_fs, "y_fs : ", y_fs)
print("------------------------------------3----------------------------------------------------")
image = cv2.imread(imageName)
leastSquareVP(image, filterHoughLinesstd,"-filtered-")
print("------------------------------------33----------------------------------------------------")

image = cv2.imread(imageName)
IMAGE_HEIGHT = image.shape[0]
IMAGE_WIDTH = image.shape[1]

intersections_alt = intersectionofmultipleLines (filterHoughLinesstd, IMAGE_HEIGHT, IMAGE_WIDTH)
print("intersections_alt", intersections_alt)
cNum = 10
if cNum > len(intersections_alt):
    cNum = len(intersections_alt)
freq = Counter(tuple(item) for item in intersections_alt).most_common(cNum)
for f in freq:
    print(f[0])
    i_h = int(image.shape[0]) 
    i_w = int(image.shape[1])
    cv2.circle(image,f[0] ,int(i_w/60),(random. randint(0,255),random. randint(0,255),random. randint(0,255)),3)
    cv2.imwrite("maxIntersectpoint_alt.jpg", image)

print("------------------------------------4----------------------------------------------------")

# ##################################################################################################
# image = cv2.imread(imageName)

# homoLinesstd = computeHomologousRepresentation(filterHoughLinesstd)
# lines_inter1 = []
# for j in range(len(homoLinesstd) - 1):
#         for k in range(j+1, len(homoLinesstd)):
#             lines_inter1.append((homoLinesstd[j], homoLinesstd[k]))

# list_of_lists = np.array([item[0] for item in lines_inter1])
# new_int, intersections = findIntersectionfromHomogenousRepresentationsofLines(list_of_lists)
# cNum = 10
# if cNum > len(intersections):
#     cNum = len(intersections)
# freq = Counter(tuple(item) for item in intersections).most_common(cNum)
# for f in freq:
#     print(f[0])
#     i_h = int(image.shape[0]) 
#     i_w = int(image.shape[1])
#     cv2.circle(image,(int(f[0][0]), int(f[0][1])) ,int(i_w//60),(random. randint(0,255),random. randint(0,255),random. randint(0,255)),3)
#     cv2.imwrite("maxIntersectpoint_std.jpg", image)
# print("------------------------------------5----------------------------------------------------")

